In [1]:
import tensorflow as tf
import tensorflow_model_optimization as tfmot
from utils import *
import numpy as np
import keras
from keras.models import Model
import re

Load model from h5 files.

In [2]:
with tfmot.quantization.keras.quantize_scope():
    model = tf.keras.models.load_model('./model/MobileNet_quantized_weights.h5')
# model = keras.models.load_model('./model/MobileNet_quantized.h5')

In [12]:
# model.summary()

In [3]:
# model.get_config()
layers_name_list = []
for i in model.get_config()['layers']:
    layers_name_list.append(i['config']['name'])

In [6]:
# print(layers_name_list)

conv_layer_name = [j for j in layers_name_list if (j.find('conv2d') != -1 and j.find('conv2d_input') == -1)]
BN_layer_name = [j for j in layers_name_list if (j.find('batch_normalization') != -1 )]
print(conv_layer_name)
print(BN_layer_name)

['quant_conv2d', 'quant_depthwise_conv2d', 'quant_conv2d_1', 'quant_depthwise_conv2d_1', 'quant_conv2d_2', 'quant_depthwise_conv2d_2', 'quant_conv2d_3', 'quant_depthwise_conv2d_3', 'quant_conv2d_4', 'quant_depthwise_conv2d_4', 'quant_conv2d_5', 'quant_depthwise_conv2d_5', 'quant_conv2d_6', 'quant_depthwise_conv2d_6', 'quant_conv2d_7', 'quant_depthwise_conv2d_7', 'quant_conv2d_8', 'quant_depthwise_conv2d_8', 'quant_conv2d_9', 'quant_depthwise_conv2d_9', 'quant_conv2d_10', 'quant_depthwise_conv2d_10', 'quant_conv2d_11', 'quant_depthwise_conv2d_11', 'quant_conv2d_12', 'quant_depthwise_conv2d_12', 'quant_conv2d_13']
['quant_batch_normalization', 'quant_batch_normalization_1', 'quant_batch_normalization_2', 'quant_batch_normalization_3', 'quant_batch_normalization_4', 'quant_batch_normalization_5', 'quant_batch_normalization_6', 'quant_batch_normalization_7', 'quant_batch_normalization_8', 'quant_batch_normalization_9', 'quant_batch_normalization_10', 'quant_batch_normalization_11', 'quant_

In layer `quant_conv2d`, the shape is (3, 3, 3, 32), which means (K_row, K_col, K_channel, N_K).
In this part, the original array should be converted to the shape(32, 3, 9).

In [15]:
array = model.get_layer('quant_conv2d').get_weights()[0]
print(array, array.shape)

[-0.00144564 -0.00335018  0.00726821 -0.00252246 -0.00942446 -0.00140176
 -0.00039517  0.00083479 -0.00012807 -0.00197633 -0.0031901  -0.00038116
  0.00912432 -0.0026863  -0.00146507 -0.00256533  0.00131126 -0.00675334
  0.00377415  0.00403005  0.01148594 -0.00048946  0.00797359  0.00765559
  0.00112162  0.00480905  0.00938235  0.0052577   0.00260402 -0.00054896
  0.00035515 -0.00767266] (32,)


In [16]:
f = open('config.h', 'w', encoding='utf-8')

In [17]:
# array.shape
# f.write('const float quant_conv2d_weight[32][3][9] = {\n')
# for l in range(32):
#     f.write('    {\n')
#     for k in range(3):
#         f.write("        { ")
#         for j in range(3):
#             for i in range(3):
#                 f.write(str(array[j][i][k][l])  + ', ')
#         f.write('}, \n')
#     f.write('    }, \n')
# f.write('}\n')

In [18]:
f.close()

In `depthwise conv2d`, the shape is (3, 3, 32, 1), which means (K_row, K_col, K_channel, None).
In this part, the original array should be converted to the shape (32, 3, 3) which means (K_channel, K_row, K_col).

In [19]:
array = model.get_layer('quant_depthwise_conv2d').get_weights()[1]
# print(array)
print('const float ' + 'quant_depthwise_conv2d' + '_weight[32][3][3] = {')
for k in range(32):
    print('    {')
    for i in range(3):
        print('        {', end='')
        for j in range(3):
            print(array[i][j][k][0], end=', ')
        print('},')
    print('    }, ')
print('}')

const float quant_depthwise_conv2d_weight[32][3][3] = {
    {
        {0.11912161, 0.2343216, 0.054199446, },
        {0.00044625712, 0.1650363, -0.13595125, },
        {-0.12612528, 0.058032405, -0.11137912, },
    }, 
    {
        {0.111497276, -0.06944291, -0.0023079608, },
        {0.30059606, 0.098657556, -0.0823326, },
        {0.06112216, 0.058675114, 0.01291736, },
    }, 
    {
        {-0.054936435, 0.11391866, 0.12827961, },
        {-0.13914777, -0.043504246, 0.074759245, },
        {-0.14660038, -0.14813481, -0.07772566, },
    }, 
    {
        {-0.13258699, -0.100685, -0.049561724, },
        {0.14407775, 0.071677, 0.017693995, },
        {-0.036664583, -0.02450803, 0.1273828, },
    }, 
    {
        {0.12860042, -0.120174155, -0.13505404, },
        {0.13529323, 0.07736491, -0.17988575, },
        {0.124753095, 0.020296425, 0.07757967, },
    }, 
    {
        {0.046516065, 0.024652585, 0.09127982, },
        {-0.072362654, -0.017119847, -0.07916848, },
        {0.044

In pointwise conv, such as `quant_conv2d_1`, the shape is (1, 1, 32, 64), which means (1, 1, F_input_channel, N_K).
In this part, the original array should be converted to the shape (64, 32) which means (N_K, F_input_channel)

In [20]:
array = model.get_layer('quant_conv2d_1').get_weights()[1]
print(array.shape[3])
# print(array)
print('const float ' + 'quant_conv2d_1' + '_weight[64][32] = {')
for j in range(64):
    print('    {', end='')
    for i in range(32):
        for k in range(1):
            for l in range(1):
                print(array[k][l][i][j], end=', ')
    print('}, ')
print('}')

64
const float quant_conv2d_1_weight[64][32] = {
    {0.0587508, -0.2986698, 0.22428569, 0.087568544, -0.081387654, -0.0046160202, -0.01828102, 0.22399734, 0.30778614, -0.21299557, -0.118106045, -0.17134304, -0.11968391, -0.24803777, -0.2187094, -0.09526928, -0.048942007, -0.1573715, -0.20544817, 0.029429048, 0.0602412, 0.13310412, -0.26715288, 0.15710042, -0.197392, 0.095137015, -0.25836784, 0.09142265, 0.10181463, -0.048025075, -0.20683494, 0.22096303, }, 
    {0.17993577, -0.10948905, -0.06378293, 0.108827, 0.12239433, -0.23134917, -0.13419303, 0.15927573, -0.054110304, 0.040498093, -0.23494162, 0.14719108, 0.11150406, 0.33218408, -0.20069182, -0.16113184, 0.16359732, 0.13167556, -0.24334101, -0.21652123, 0.21906078, -0.04599223, 0.114636116, 0.09561855, 0.31893644, 0.093320094, 0.3354819, 0.0999434, 0.19006161, 0.2235626, 0.43219525, -0.13755862, }, 
    {-0.05645652, 0.03422675, -0.03170307, -0.15542062, -0.09463051, 0.057950694, 0.18319179, -0.08886807, 0.22122394, 0.12906651, -0

In fully connected layer (`Dense`), the shape is (1024, 43).
In this part, the original array should be converted to the shape (43, 1024).

In [7]:
array = model.get_layer('quant_dense').get_weights()[1]
print(array.shape)
print(array)

for j in range(43):
    for i in range(1024):
        # print(array[i][j])

SyntaxError: unexpected EOF while parsing (<ipython-input-7-268ce4db9542>, line 8)

In BN layer such as `quant_batch_normalization`, the shape is(32, 32, 32, 32) which the number equals the channel number.
All the parameters saved in BN layers is as the array (gamma, beta, mean, variance).

In [15]:
array = model.get_layer('quant_batch_normalization').get_weights()[0]
print(array.shape)
print(array)

(32,)
[1.1355538  0.93469393 0.95581883 0.63997316 1.1997472  0.6730653
 1.1607682  1.0886629  0.9873546  1.0239314  1.1888553  0.94607323
 0.99668926 1.0437193  0.9545999  0.8541228  0.8129851  0.95708877
 0.8386617  0.9914782  0.9822208  1.1182891  0.8842698  1.1580402
 0.93770945 1.0244526  0.8873894  1.0720729  0.9194398  1.0007893
 1.0383357  1.0590619 ]


From above, we can make a function to convert the model to c file.

This function is defined in file [utils.py](utils.py)

In [ ]:
convert_h5_to_c(conv_layer_name, model)

NameError: name 'convert_h5_to_c' is not defined

For quantized model, the layer number is as shown below:
```
32 29 -> Traditional Convolution

// Dw and Pw Convolution
34 28
36 17
38 9
40 8
42 7
44 6
46 5
48 4
50 3
52 27
54 26
56 25
58 24
60 23
62 22
64 21
66 20
68 19
70 18
72 16
74 15
76 14
78 13
80 12
82 11
84 10

88 30 -> Fully Connection
```

If you want to get the parameters of the model, you should get the quantized model first.

In [3]:
lite_model = tf.lite.Interpreter(model_path='./model/newer/MobileNet.tflite')
lite_model.allocate_tensors()

For Traditional Convolution (layer 32 and layer 29), the shape of the weight is `(32, 3, 3, 3)`.

In [3]:
f = open('D:/vivado_projects/ip_repo/MobileNet_1_0/hdl/hex/t_conv.hex', 'w')

layer = lite_model.get_tensor(32)
# layer.shape
# print(layer)

strl = []

for c in range(32):
    s = ""
    for i in range(3):
        for j in range(3):
            for k in range(3):
                # s += dec2INT8InHex(layer[c][j][k][i])
                s += dec2INT8InHex(layer[c][i][j][k])
                # print(layer[c][j][k][i])
    strl.append(s)
# print(strl)

l = []
tt = []

ss = ''

for i in strl:
    ss += i

res = re.findall(r'.{8}', ss)

# print(np.asarray(res).shape)

for i in res:
    f.write(i)
    f.write('\n')

layer = lite_model.get_tensor(29)

# print(layer)

for i in layer:
    f.write(dec2INT32InHex(i))
    f.write('\n')

f.close()

For depthwise Convolution (layer 34 and layer 28), the shape of the weight is `(1, 3, 3, 32)`.

In [4]:
f = open('D:/vivado_projects/ip_repo/MobileNet_1_0/hdl/hex/dw_2_conv.hex', 'w')

w_layer = lite_model.get_tensor(38)
b_layer = lite_model.get_tensor(9)
M = [0x00062169, 0x0004f757, 0x0006285b, 0x00047747, 0x0004e61c, 0x0002a2bb, 0x00044d29, 0x000631e4, 0x0002ff18, 0x000347a8, 0x0002d613, 0x00029ebb, 0x0004906a, 0x00069714, 0x0003ffe9, 0x0003f652, 0x00035da4, 0x00032c66, 0x00032f47, 0x00023d15, 0x00039972, 0x0004672e, 0x0003d53a, 0x0004d41e, 0x0005ce7b, 0x0002f68c, 0x00037500, 0x0006f59f, 0x000363d3, 0x0006d1b8, 0x0003d2da, 0x000514b3, 0x0006c24c, 0x0003e6a3, 0x000c219e, 0x0004ff29, 0x0003d323, 0x00028a8f, 0x00088a47, 0x00036a08, 0x0003656d, 0x0006b308, 0x0002aa0d, 0x0004ceef, 0x000310aa, 0x000dff89, 0x00039ace, 0x0004dd2e, 0x00027623, 0x000e92fb, 0x00038d4e, 0x00066c41, 0x00066e7a, 0x00033bcc, 0x00078edf, 0x000466d2, 0x00045698, 0x000632d6, 0x00041a57, 0x0003f542, 0x000da131, 0x0002e485, 0x0004b503, 0x00034406]

strl = []

s = ""

for n in range(int(64 / 32)):
    s = ""
    for c in range(32):
        for i in range(3):
            for j in range(3):
                # s += dec2INT8InHex(layer[c][j][k][i])
                s += dec2INT8InHex(w_layer[0][i][j][n * 32 + c])
                # print(layer[c][j][k][i])

    res = re.findall(r'.{8}', s)

    for i in res:
        f.write(i)
        f.write('\n')

    # print(layer)

    for i in range(32):
        f.write(dec2INT32InHex(b_layer[n * 32 + i]))
        f.write('\n')

    for i in range(32):
        f.write(dec2INT32InHex(M[n * 32 + i]))
        f.write('\n')

f.close()

For pointwise Convolution, (layer 36 and layer 17), the shape of the weight is `(64, 1, 1, 32)`.

In [26]:
M = [
    "0001a2b2",
    "0001744e",
    "0001cec0",
    "0002235d",
    "00024e26",
    "0001bbd4",
    "00016148",
    "0002ac4a",
    "00026aea",
    "0002fffc",
    "00021d3f",
    "0001e676",
    "00025cf4",
    "000162f5",
    "00020e76",
    "0003003d",
    "000274f9",
    "00014acb",
    "00020632",
    "0001c9eb",
    "0002832a",
    "000180f2",
    "00021ea0",
    "000164ab",
    "0002b102",
    "0002193f",
    "00016bcb",
    "00014249",
    "00033aa7",
    "0000ed94",
    "000114a3",
    "00023276",
    "0001a53c",
    "0002f980",
    "00019176",
    "0001476e",
    "000311cb",
    "0001b5bf",
    "0001b677",
    "0001f3a2",
    "0001be12",
    "00025829",
    "0002a13d",
    "00034176",
    "000238b1",
    "0000e41f",
    "000206a9",
    "00017a3b",
    "00030f1d",
    "0000c916",
    "0001494b",
    "0001c858",
    "0002059d",
    "0001c343",
    "0001b886",
    "00024d27",
    "000315d3",
    "00024b26",
    "00029fa3",
    "00024946",
    "0000e551",
    "00019cac",
    "00037f60",
    "0002e9f8"
]

N = 64
channel = 32

f = open('D:/vivado_projects/ip_repo/MobileNet_1_0/hdl/hex/pw_conv.hex', 'w')

layer = lite_model.get_tensor(36)
bias = lite_model.get_tensor(17)

strl = []

s = ""

for n in range(int(N / 32)):
    st = ""
    tl = []
    gl = []
    g = []
    ts = ""
    gt = []
    s = ""
    for c in range(int(channel / 32)):
        for nn in range(n * 32, n * 32 + 32):
            for cc in range(c * 32, c * 32 + 32):
                tl.append(dec2INT8InHex(layer[nn,0,0,cc]))
            g = []
            for i in range(int(len(tl) / 4)):
                gl = []
                for temp in range(4):
                    gl.append(tl[i * 4 + temp])
                gl.reverse()
                g.append(gl)

        for gs in g:
            ts = ""
            for i in range(4):
                ts += gs[i]
            gt.append(ts)

            # t_r = re.findall(r'.{8}', st)

    for bb in range(n * 32, n * 32 + 32):
        s += dec2INT32InHex(bias[bb])
    for mm in range(n * 32, n * 32 + 32):
        s += M[mm]

    res = re.findall(r'.{8}', s)

    res = gt + res

    for i in res:
        f.write(i)
        f.write('\n')


f.close()